#Load

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

DATA_FOLDER = "../../DATA/"

In [2]:
df = pd.read_parquet(DATA_FOLDER + '/PROCESSING/train_models_merged_features.parq')
df

,attack_time,watcher_country,watcher_as_num,watcher_as_name,attacker_country,attacker_as_num,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum,label,protocol,joined_countries,port_count,HAS_TCP?,HAS_UDP?,risky_ports
0,1.762414,US,-0.091011,PEGTECHINC,FR,-0.502362,OVH SAS,bruteforce,-0.958130,95867,0,ssh,FR:US,3,False,False,True
1,1.409984,PL,-0.503736,home.pl S.A.,FR,-0.502362,OVH SAS,spam,-0.101544,115212,0,http,FR:PL,4,False,False,True
2,0.235214,US,-0.090606,IMH-IAD,IL,-0.596048,Bezeq- THE ISRAEL TELECOMMUNICATION CORP. LTD.,exploit,0.969242,106785,0,http,IL:US,0,False,False,False
3,-1.291987,US,-0.311877,LIQUIDWEB,US,-0.217067,"Alibaba US Technology Co., Ltd.",scan,-0.791761,6738,0,http,US:US,0,False,False,False
4,0.587645,DE,-0.153113,IP-Projects GmbH & Co. KG,FR,-0.450669,Host Europe GmbH,exploit,-0.913167,117158,0,http,FR:DE,6,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20007007,-0.704602,DE,-0.286743,Neue Medien Muennich GmbH,ID,-0.587111,PT Telekomunikasi Indonesia,exploit,-0.735038,173996,0,http,ID:DE,0,False,False,False
20007008,-1.644418,DK,-0.121954,One.com A/S,US,-0.583528,MICROSOFT-CORP-MSN-AS-BLOCK,spam,0.228971,35213,0,http,US:DK,17,False,False,True
20007009,-0.352171,NL,-0.238176,CJ2 Hosting B.V.,SG,0.741896,Contabo Asia Private Limited,scan,-0.569887,87392,0,http,SG:NL,1,False,False,True
20007010,-1.761895,FR,1.379804,GANDI SAS,SG,-0.524284,DIGITALOCEAN-ASN,scan,-0.904705,22588,0,http,SG:FR,0,False,False,False


In [3]:
unique_ips = df['attacker_ip_enum'].unique()

TRAIN, TEST = train_test_split(unique_ips, test_size=0.25, random_state=0)

TRAIN = df[df['attacker_ip_enum'].isin(TRAIN)]
TEST = df[df['attacker_ip_enum'].isin(TEST)]
del unique_ips
del df

In [4]:
TEST, VAL = train_test_split(TEST, test_size=0.3, random_state=0)

In [5]:
TEST1P = (TEST["label"].sum() / len(TEST)) * 100
TRAIN1P = (TRAIN["label"].sum() / len(TRAIN)) * 100
VAL1P = (VAL["label"].sum() / len(VAL)) * 100


print(f"Percentage of 1s in Dataset 1: {TEST1P:.2f}%")
print(f"Percentage of 1s in Dataset 2: {TRAIN1P:.2f}%")
print(f"Percentage of 1s in Dataset 3: {VAL1P:.2f}%")

del TEST1P
del TRAIN1P
del VAL1P


XIPS = set(TEST["attacker_ip_enum"]).intersection(set(TRAIN["attacker_ip_enum"]))
XIPS2 = set(VAL["attacker_ip_enum"]).intersection(set(TRAIN["attacker_ip_enum"]))

if len(XIPS) == 0 and len(XIPS2) == 0:
    print("IPs do not cross between datasets.")
else:
    print("IPs cross between datasets.")

del XIPS2
del XIPS

print(f"Size of Dataset 1: {len(TEST)}")
print(f"Size of Dataset 2: {len(TRAIN)}")
print(f"Size of Dataset 3: {len(VAL)}")



Percentage of 1s in Dataset 1: 1.03%
Percentage of 1s in Dataset 2: 2.06%
Percentage of 1s in Dataset 3: 1.02%
IPs do not cross between datasets.
Size of Dataset 1: 7333064
Size of Dataset 2: 29538218
Size of Dataset 3: 3142742


In [6]:
TEST.to_parquet(DATA_FOLDER + '/PROCESSING/TEST_SPLIT.parq', index=True)
VAL.to_parquet(DATA_FOLDER + '/PROCESSING/VAL_SPLIT.parq', index=True)
del TEST
del VAL

In [7]:
TRAIN.loc[:, 'Index_Column'] = TRAIN.reset_index().index

C:\Users\Francisco\AppData\Local\Temp\ipykernel_7844\3007962949.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TRAIN.loc[:, 'Index_Column'] = TRAIN.reset_index().index


In [8]:
print("29538218 rows × 18 columns")
len(TRAIN['Index_Column'].unique())

29538218 rows × 18 columns


29538218

In [9]:
rus = RandomUnderSampler(sampling_strategy=0.4, random_state=0)
x = TRAIN[["Index_Column", "attacker_ip_enum"]]
y = TRAIN['label']
X_train_resampled, y_train_resampled = rus.fit_resample(x, y)
TRAIN_aux = pd.concat([pd.DataFrame(X_train_resampled), y_train_resampled], axis=1)
del x
del y
del X_train_resampled
del y_train_resampled
del rus

In [10]:
print((TRAIN_aux["label"].sum() / len(TRAIN_aux)) * 100)
print(len(TRAIN_aux['Index_Column'].unique()))
print(f"Size of Dataset 2: {len(TRAIN_aux)}")

28.571435277086856
2130397
Size of Dataset 2: 2130397


In [11]:
TRAIN_aux

,Index_Column,attacker_ip_enum,label
19265251,28989933,135778,0
13857975,10229542,184076,0
19454751,29129990,45508,0
16948573,12511243,109140,0
4268348,17920228,89771,0
...,...,...,...
20006654,29537963,2841,1
20006661,29537966,157556,1
20006814,29538077,123714,1
20006978,29538193,167107,1


In [12]:
common_values = TRAIN[TRAIN['Index_Column'].isin(TRAIN_aux['Index_Column'])]['Index_Column'].unique()
len(common_values)

2130397

In [13]:
TRAIN = TRAIN[TRAIN['Index_Column'].isin(common_values)]
TRAIN

,attack_time,watcher_country,watcher_as_num,watcher_as_name,attacker_country,attacker_as_num,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum,label,protocol,joined_countries,port_count,HAS_TCP?,HAS_UDP?,risky_ports,Index_Column
45,-0.352171,AU,-0.076617,Entity Data Pty Limited,CA,-0.341899,AMANAHA-NEW,spam,-0.402299,16557,1,http,CA:AU,1,False,False,False,33
95,1.292507,FR,-0.469632,OVH SAS,FI,-0.416613,Hetzner Online GmbH,spam,-0.761499,175622,0,http,FI:FR,1,False,False,True,72
108,-0.352171,US,-0.003973,PRIVATESYSTEMS,LU,-0.132298,PONYNET,spam,-0.699756,84916,0,http,LU:US,3,False,False,True,82
112,-0.352171,US,-0.090606,IMH-IAD,US,-0.549383,HRTC,scan,-0.092007,85069,0,http,US:US,0,False,False,False,86
124,0.705122,US,-0.160335,Hostinger International Limited,US,-0.466781,NETWORK-SOLUTIONS-HOSTING,scan,-0.174904,195272,0,http,US:US,20,False,False,True,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20006979,1.057553,US,-0.333226,AS-30083-GO-DADDY-COM-LLC,US,-0.402688,DREAMHOST-AS,exploit,-0.446258,75769,1,http,US:US,2,False,False,True,29538194
20006996,0.705122,DE,-0.124927,Contabo GmbH,US,1.432369,Suisse Limited,bruteforce,-0.567593,153844,0,ssh,US:DE,0,False,False,False,29538204
20006998,-0.704602,US,-0.311877,LIQUIDWEB,IR,-0.059960,Pars Online PJS,spam,-0.433063,151482,0,http,IR:US,15,False,False,True,29538206
20007005,1.292507,US,3.302596,GO-DADDY-COM-LLC,NL,-0.174004,Global Layer B.V.,spam,0.621731,162497,0,http,NL:US,2,False,False,False,29538211


# SAVE


In [14]:
TRAIN.to_parquet(DATA_FOLDER + '/PROCESSING/TRAIN_SPLIT.parq', index=True)